In [35]:
import pandas as pd

datapath = 'universe_8' 
galaxDists = pd.read_csv('Galaxy_Distances.csv', sep = ' ')
xrayData = pd.read_csv('Flash_Data.csv', sep = ' ')
print(galaxDists)

                                                  Name   PL_distance  \
0    Cheese-0.12734780487804878Pizza-12.30588292682...  18812.991250   
1    Cheese-0.5590729881823298Pizza-15.085296567248...  17547.473092   
2    Cheese-0.656434251968504Pizza-5.26184074803149...  26333.855891   
3                          Cheese-00.1Pizza-12.3Galaxy  18812.991250   
4                          Cheese-00.6Pizza-15.1Galaxy  17547.473092   
..                                                 ...           ...   
244                                 X038.5-Y029.5-N583  28566.185085   
245                                 X039.6-Y-10.7-N639   3671.813606   
246                                 X041.1-Y032.0-N398           NaN   
247                                 X041.5-Y-04.3-N229   1034.291533   
248                                 X041.7-Y036.3-N523  35178.668659   

          PL_unc    MS_distance      MS_unc  
0    1457.117253   96593.917273  125.152810  
1    1527.809894  104671.966989  135.619211

In [47]:
import numpy as np
import re

equats, polars, dists = np.zeros(len(galaxDists)), np.zeros(len(galaxDists)), np.zeros(len(galaxDists))


    

for i in range(len(galaxDists)):
    name = galaxDists['Name'][i]
    # trying to get the x and y pos of the galaxies from their names
    # but x and y is inconsistent in the name
    # two options: use regex but i tried and not working
    # OR do back to where the galaxies are made and print out their x and y as well as the other data in csv
    # they were made in galaxy_finder
    
    equats[i] = name[7:24]
    
    
    polars[i] = name[31:49]
    dists[i] = galaxDists['MS_distance'][i]

ValueError: could not convert string to float: '.261840748031496Ga'

In [ ]:
closeflashes = xrayData.loc[xrayData['Photon-Count'] > 1e6]
print(closeflashes)

In [ ]:
lumins = []
delta = 1.5
for i, flash in closeflashes.iterrows():
    for j, equat in enumerate(equats):
        if (flash['Equatorial'] - delta <= equat <= flash['Equatorial'] + delta
           ) and (flash['Polar'] - delta <= polars[j] <= flash['Polar'] + delta):
            lumin = flash['Photon-Count'] * 4 * np.pi * dists[j]**2
            lumins.append(lumin)
lumins = np.array(lumins)
print(lumins)

In [ ]:
base_lumin = np.mean(lumins)
xraydists = np.zeros(len(xrayData))
for i, flash in xrayData.iterrows():
    xraydists[i] = np.sqrt(base_lumin / (4 * np.pi * flash['Photon-Count']))


In [ ]:
# created this data in finding_galaxies.ipynb
distGalaxData = pd.read_csv(datapath + '/Distant_Galaxy_Data.csv')
speeds = np.zeros(len(xrayData))
delta = 0.5
for i, flash in xrayData.iterrows():
    for j, equat in enumerate(distGalaxData['Equatorial']):
        if (flash['Equatorial'] - delta <= equat <= flash['Equatorial'] + delta
           ) and (flash['Polar'] - delta <= distGalaxData['Polar'][j] <= flash['Polar'] + delta):
            speeds[i] = distGalaxData['RadialVelocity'][j]

In [ ]:
import matplotlib.pyplot as plt

grad, yint = np.polyfit(xraydists, speeds, 1)
xvals = np.array([0, max(xraydists)])

print(f"m = {round(grad * 1e6, 2)} km/s/Mpc; y-int = {round(yint, 2)} km/s")

fig, ax = plt.subplots()
ax.scatter(xraydists, speeds)
ax.plot(xvals, grad * xvals + yint, 'r-')
ax.set_xlabel("Distance (pc)");
ax.set_ylabel("Radial Velocity (km/s)");

In [ ]:
distGalaxData['Distance'] = (distGalaxData['RadialVelocity'].to_numpy() - yint) / grad

homogeneous universe code
